# Troubleshooting Datasets Notebook
tinkering with dataset connections behaving unexpectedly

In [2]:
# Fix MTA Art Station Matching
import requests
import pandas as pd
from difflib import SequenceMatcher
import re

print("TROUBLESHOOTING MTA ART DATASET")
print("=" * 60)

# fetch both datasets
ridership_url = "https://data.ny.gov/resource/wujg-7c2s.json"
art_url = "https://data.ny.gov/resource/4y8j-9pkd.json"

# get sample ridership stations
ridership_response = requests.get(ridership_url, params={'$limit': 100})
ridership_data = ridership_response.json()
ridership_df = pd.DataFrame(ridership_data)

# get all art data (should be small dataset)
art_response = requests.get(art_url, params={'$limit': 1000})
art_data = art_response.json()
art_df = pd.DataFrame(art_data)

print(f"ridership stations sample: {ridership_df['station_complex'].nunique()}")
print(f"art installations: {len(art_df)}")

# examine naming patterns
print("\nridership station names (first 5):")
for station in ridership_df['station_complex'].unique()[:5]:
    print(f"  {station}")

print("\nart station names (first 5):")
for station in art_df['station_name'].unique()[:5]:
    print(f"  {station}")

# try different matching strategies
def normalize_station_name(name):
    """normalize station names for matching"""
    if pd.isna(name):
        return ""
    # remove line info in parentheses
    name = re.sub(r'\([^)]*\)', '', name)
    # remove common suffixes
    name = name.replace(' Station', '').replace(' - ', '-')
    # normalize spaces and case
    name = ' '.join(name.split()).strip().lower()
    return name

# normalize both
ridership_df['station_normalized'] = ridership_df['station_complex'].apply(normalize_station_name)
art_df['station_normalized'] = art_df['station_name'].apply(normalize_station_name)

# try exact matches after normalization
matches = set(ridership_df['station_normalized']) & set(art_df['station_normalized'])
print(f"\nmatches after normalization: {len(matches)}")
if matches:
    print("example matches:")
    for match in list(matches)[:5]:
        print(f"  {match}")

TROUBLESHOOTING MTA ART DATASET
ridership stations sample: 87
art installations: 381

ridership station names (first 5):
  Woodhaven Blvd (M,R)
  36 St (M,R)
  Union St (R)
  Jamaica-Van Wyck (E)
  Greenpoint Av (G)

art station names (first 5):
  Clark St
  125 St
  Astor Pl
  Kings Hwy
  Newkirk Av

matches after normalization: 50
example matches:
  181 st
  jackson av
  christopher st-sheridan sq
  145 st
  cathedral pkwy


In [3]:
# Fuzzy Matching for Stations
print("\nFUZZY MATCHING ATTEMPT")
print("-" * 40)

# for each art station, find best match in ridership
def find_best_match(art_station, ridership_stations, threshold=0.8):
    """find best matching station using fuzzy matching"""
    best_match = None
    best_score = 0
    
    for ride_station in ridership_stations:
        score = SequenceMatcher(None, art_station, ride_station).ratio()
        if score > best_score and score >= threshold:
            best_score = score
            best_match = ride_station
    
    return best_match, best_score

# test on a few art stations
ridership_stations = ridership_df['station_normalized'].unique()
art_sample = art_df['station_normalized'].unique()[:10]

print("fuzzy matching results (threshold=0.8):")
for art_station in art_sample:
    if art_station:  # skip empty
        match, score = find_best_match(art_station, ridership_stations)
        if match:
            print(f"  '{art_station}' -> '{match}' (score: {score:.2f})")


FUZZY MATCHING ATTEMPT
----------------------------------------
fuzzy matching results (threshold=0.8):
  '125 st' -> '125 st' (score: 1.00)
  'kings hwy' -> 'kings hwy' (score: 1.00)
  '23 st' -> '23 st' (score: 1.00)
  '50 st' -> '55 st' (score: 0.80)
  '86 st' -> '36 st' (score: 0.80)


In [4]:
# Alternative - Use Coordinates for Matching
print("\nCOORDINATE-BASED MATCHING")
print("-" * 40)

# check if art data has coordinates
print("art columns:", list(art_df.columns))

# look for location columns
location_cols = [col for col in art_df.columns if 'lat' in col.lower() or 'lon' in col.lower() or 'geo' in col.lower() or 'location' in col.lower()]
print(f"potential location columns: {location_cols}")


COORDINATE-BASED MATCHING
----------------------------------------
art columns: ['agency', 'station_name', 'line', 'artist', 'art_title', 'art_date', 'art_material', 'art_description', 'art_image_link', 'station_normalized']
potential location columns: []


In [5]:
# Get Weather Data Without API Key
print("\n" + "=" * 60)
print("WEATHER DATA ALTERNATIVES")
print("=" * 60)

# option 1: use historical weather from NOAA (free, no key needed for historical)
noaa_url = "https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/2024/"
print("option 1: noaa historical data")
print(f"  url pattern: {noaa_url}[STATION_ID].csv")
print("  central park station: 72505394728")
print("  laguardia: 72503014732")

# test central park data
central_park_2024 = "https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/2024/72505394728.csv"
try:
    # just test if accessible
    response = requests.head(central_park_2024)
    if response.status_code == 200:
        print(f"  ✓ central park weather data accessible!")
        
        # fetch sample
        data_response = requests.get(central_park_2024)
        lines = data_response.text.split('\n')[:5]
        print("  sample data:")
        for line in lines[:2]:
            print(f"    {line[:100]}...")
except Exception as e:
    print(f"  ✗ error: {e}")

# option 2: use nyc open data weather
print("\noption 2: nyc open data weather datasets")
weather_urls = {
    "central_park_weather": "https://data.cityofnewyork.us/resource/bqiq-cu78.json",  # historical
}

for name, url in weather_urls.items():
    try:
        response = requests.get(url, params={'$limit': 5})
        if response.status_code == 200:
            data = response.json()
            if data:
                print(f"  ✓ {name} accessible")
                print(f"    columns: {list(data[0].keys()) if data else 'no data'}")
    except:
        print(f"  ✗ {name} not accessible")


WEATHER DATA ALTERNATIVES
option 1: noaa historical data
  url pattern: https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/2024/[STATION_ID].csv
  central park station: 72505394728
  laguardia: 72503014732
  ✓ central park weather data accessible!
  sample data:
    "STATION","DATE","LATITUDE","LONGITUDE","ELEVATION","NAME","TEMP","TEMP_ATTRIBUTES","DEWP","DEWP_ATT...
    "72505394728","2024-01-01","40.77898","-73.96925","42.7","NY CITY CENTRAL PARK, NY US","  42.5","24"...

option 2: nyc open data weather datasets
  ✓ central_park_weather accessible
    columns: ['full_complaint_id', 'complaint_year_number', 'month_number', 'record_create_date', 'complaint_precinct_code', 'patrol_borough_name', 'county', 'law_code_category_description', 'offense_description', 'pd_code_description', 'bias_motive_description', 'offense_category']


In [6]:
# Cell 5: NASA Black Marble - Direct Tile Access
print("\n" + "=" * 60)
print("NASA BLACK MARBLE - TESTING DIRECT ACCESS")
print("=" * 60)

# nasa gibs provides tiles without authentication for certain layers
# test different date formats and zoom levels

base_url = "https://gibs.earthdata.nasa.gov/wmts/epsg4326/best"
layer = "VIIRS_Black_Marble"

# nyc tile coordinates at different zoom levels
test_configs = [
    {"date": "2023-01-01", "zoom": "8", "row": "77", "col": "302"},  # zoom 8
    {"date": "2022-01-01", "zoom": "7", "row": "38", "col": "151"},  # zoom 7
    {"date": "2021-01-01", "zoom": "6", "row": "19", "col": "75"},   # zoom 6
]

print("testing nasa black marble tile access:")
for config in test_configs:
    tile_url = f"{base_url}/{layer}/default/{config['date']}/GoogleMapsCompatible_Level{config['zoom']}/{config['zoom']}/{config['col']}/{config['row']}.png"
    
    try:
        response = requests.head(tile_url, timeout=5)
        if response.status_code == 200:
            print(f"  ✓ zoom {config['zoom']}: accessible")
            print(f"    url: {tile_url}")
            break
        else:
            print(f"  ✗ zoom {config['zoom']}: {response.status_code}")
    except Exception as e:
        print(f"  ✗ zoom {config['zoom']}: {str(e)[:50]}")

# alternative: use nasa worldview snapshots (pre-rendered, no auth)
worldview_url = "https://worldview.earthdata.nasa.gov/config/wv.json"
print(f"\nalternative: nasa worldview (pre-rendered snapshots)")
print(f"  config url: {worldview_url}")

try:
    response = requests.get(worldview_url, timeout=5)
    if response.status_code == 200:
        print("  ✓ worldview config accessible")
        print("  can use worldview snapshots for specific dates")
except:
    print("  ✗ worldview not accessible")

# Cell 6: Alternative Data Sources
print("\n" + "=" * 60)
print("ADDITIONAL NYC DATASETS TO CONSIDER")
print("=" * 60)

alternative_datasets = {
    "nyc_traffic_speed": "https://data.cityofnewyork.us/resource/i4gi-tjb9.json",
    "citibike_stations": "https://gbfs.citibikenyc.com/gbfs/en/station_information.json",
    "nyc_wifi_hotspots": "https://data.cityofnewyork.us/resource/yjub-udmw.json",
    "street_construction": "https://data.cityofnewyork.us/resource/i6b5-j7bu.json",
}

for name, url in alternative_datasets.items():
    try:
        response = requests.get(url if 'citibike' in name else url + "?$limit=5", timeout=5)
        if response.status_code == 200:
            data = response.json()
            if 'citibike' in name:
                print(f"✓ {name}: {len(data.get('data', {}).get('stations', []))} stations")
            else:
                print(f"✓ {name}: accessible")
                if data:
                    print(f"  columns: {list(data[0].keys())[:5]}...")
    except Exception as e:
        print(f"✗ {name}: {str(e)[:40]}")


NASA BLACK MARBLE - TESTING DIRECT ACCESS
testing nasa black marble tile access:
  ✗ zoom 8: 400
  ✗ zoom 7: 400
  ✗ zoom 6: 400

alternative: nasa worldview (pre-rendered snapshots)
  config url: https://worldview.earthdata.nasa.gov/config/wv.json
  ✓ worldview config accessible
  can use worldview snapshots for specific dates

ADDITIONAL NYC DATASETS TO CONSIDER
✗ nyc_traffic_speed: HTTPSConnectionPool(host='data.cityofnew
✓ citibike_stations: 2240 stations
✓ nyc_wifi_hotspots: accessible
  columns: ['objectid', 'borough', 'type', 'provider', 'name']...
✓ street_construction: accessible
  columns: ['the_geom', 'segmentid', 'oft', 'onstreetname', 'fromstreetname']...
